# Neo4j

This is the first part to the data exploration (see 2.2. Data Visualisation for the part 2). In this section we will begin by exploring the data using Neo4j. 

In this section we will answer one of our research questions: 
- What is the greatest risk to water supply in the US across the years?

We will additionally explore (although this was not part of our research questions):
- Which risks to water supply are most similar in terms of the number of states threatened by it?

## Load the libraries and connect to Neo4J 

In [2]:
#load the libraries
import py2neo #library used to work with Neo4j from Python
import pandas as pd

In [4]:
#connect to the server 
graph = py2neo.Graph("bolt://localhost:7687", user="neo4j", password="111")

In [3]:
#load and inspect file we will use 
neo = pd.read_csv('neo.csv')
neo.head(5)

,Unnamed: 0,Year,State,Risk,value
0,29,2021,AL,Change in land-use,1
1,55,2017,CA,Declining water quality,1
2,56,2017,CO,Declining water quality,2
3,61,2017,NV,Declining water quality,1
4,63,2017,OH,Declining water quality,1


Notice that we have an **Unnamed:0 column**, but there is no need to drop it because we will be calling only the necessary columns from neo4J.

## Degree Centrality Algorithm

In this part we are interested in exploring the greatest risk to water supply in the US overall. We will use the weighted degree centrality algorithm in Neo4j to determine the most important risk to water supply in the US and using the number of counts per state as weights. 

Let's set up the graph in Neo4j.

In [5]:
#load the data in 
#specify states and risk to be seperate nodes 
#specify whether a state highlighted a risk as being a problem through the relationship "threatened_by"
query = """
load csv with headers from "file:///neo.csv" as neo
merge (s:State{id:neo.State})
merge (r:Risk{id:neo.Risk})
merge (s)-[applies_to:threatened_by{score:toInteger(neo.value)}]->(r)"""

nodes = graph.run(query)

We can now visualise the graph in neo4j by specifying ```match(n) return n``` statement directly in neo4j. 

The graph has two nodes: the **Risk** and **State** nodes. They are connected by the **threatened_by** relationships. Each relationship starts at a **State** nodes and ends at a **Risk** node. 

With the graph set up in Neo4j, we will now project it into the graph catalog using the ```project``` execution to prepare it for the algorithm. We will use the **Risk** and **State** nodes and the **threatened_by** relationship. The graph is projected in a ```Reverse``` orientation because we are interested in the risks with greatest number of states related to it.   

In [6]:
query = """
CALL gds.graph.project(
'myGraph1',
['Risk','State'],
{
threatened_by: {
orientation: 'REVERSE', 
properties: ['score']
}
}
)
"""

nodes = graph.run(query)

We use ```stream``` execution to get the weighted frequencies for the top 6 risk.

In [7]:
query = """
CALL gds.degree.stream(
   'myGraph1',
   { relationshipWeightProperty: 'score' }
)
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS Risk, score AS weightedFreq
ORDER BY weightedFreq DESC, Risk DESC limit 6"""

nodes = graph.run(query)

Let's print the results.

In [8]:
#print the top risk to water supply 
toprisks = pd.DataFrame(nodes)
#rename columns to make it clear 
toprisks.rename({0: 'Risk', 1: 'Weighted Frequency'}, axis=1, inplace=True)
toprisks

,Risk,Weighted Frequency
0,Increased water stress or scarcity,34.0
1,Inadequate or ageing water supply infrastructure,18.0
2,Higher water prices,12.0
3,Declining water quality,12.0
4,Drought,11.0
5,Severe weather events,8.0


The table above shows that overall increased water stressed or scarcity was rated as the most problematic issue to water supply in the US across 2017 to 2021, followed by inadequate or ageing water supply infrastructure, declining water quality, higher water prices and drought. 

In the second part of the data visualisation (2.2 Data Visualisation), we will examine the risks with water supply in the US for each of the years 2017, 2018 and 2021 through the use of graphs. 

In [11]:
#delete everything for the next part 
query = """
match(n) detach delete n"""

nodes= graph.run(query)

## Similarity Algorithm 

We might also be interested in the similarities between risk in terms of the number of states specifying it as a problem (although this is not part of our research question). We can do this with the node similarity algorithm in Neo4j, which compares the **Risk** nodes based on the **States** nodes they are connected to. The more two **Risk** nodes share the same **State** nodes the more similar they are. 

Note that the algorithm will only compute similarity scores for the **Risk** nodes that have a least 1 relationship to a **State** node. In other words, if there exist no relationship between a **State** node and a **Risk** nodes then that particular **Risk** node will not be compared with other **Risk** nodes.   

Let's set up the graph in Neo4j.

In [12]:
#reload the data in 
#specify states and risk to be seperate nodes 
#specify whether a state highlighted a risk as being a problem through the relationship "threatened_by"
query = """
load csv with headers from "file:///neo.csv" as neo
merge (s:State{id:neo.State})
merge (r:Risk{id:neo.Risk})
merge (s)-[applies_to:threatened_by{score:toInteger(neo.value)}]->(r)"""

nodes2 = graph.run(query)

We can now visualise the graph in neo4j by specifying ```match(n) return n``` statement directly in neo4j. 

We have a similar graph as before. There are two nodes: the **Risk** and **State** nodes. They are connected by the **threatened_by** relationships. Each relationship starts at a **State** nodes and ends at a **Risk** node.

With the graph set up in Neo4j, we will now project it into the graph catalog using the ```project``` execution to prepare it for the algorithm. We will use the **Risk** and **State** nodes and the **threatened_by** relationship. Again we use the ```Reverse``` orientation because we are interested in similarities between the risks. 

In [13]:
query = """
CALL gds.graph.project(
    'myGraph2',
    ['Risk', 'State'],
    {
        threat: {
            orientation: 'REVERSE',
            type: 'threatened_by'
                }
            }
)
"""
nodes2 = graph.run(query)

This time the ```stream``` execution returns the similarity scores for each relationship.

In [14]:
query = """
CALL gds.nodeSimilarity.stream('myGraph2')
YIELD node1, node2, similarity
RETURN gds.util.asNode(node1).id AS Person1, gds.util.asNode(node2).id AS Person2, similarity
ORDER BY similarity DESCENDING, Person1, Person2
"""
nodes2 = graph.run(query)

In [15]:
#show the list of risk that are similar with respect to the relationship with the state nodes 
similarity = pd.DataFrame(nodes2)
#rename the columns 
similarity.rename({0: 'Risk 1', 1: 'Risk 2', 2: 'Similarity'}, axis=1, inplace=True)
similarity.head(15)

,Risk 1,Risk 2,Similarity
0,Ecosystem vulnerability,Radium in water,1.000000
1,Environmental regulations,Regulatory,1.000000
2,Radium in water,Ecosystem vulnerability,1.000000
3,Regulatory,Environmental regulations,1.000000
4,Environmental regulations,Increased water demand,0.666667
5,Increased water demand,Environmental regulations,0.666667
6,Increased water demand,Regulatory,0.666667
7,Regulatory,Increased water demand,0.666667
8,Drought,Increased water demand,0.600000
9,Increased water demand,Drought,0.600000


The table above shows the risks that are similar in terms of the **State** nodes connected to it. We see similarities between:
- Ecosystem vulnerability and radium in water, increased levels of plastic in freshwater bodies
- Environmental regulation and regulations (in general)
- Increased water demand and environmental regulations, regulations (in general) and drought
- Declining water quality and inadequate or ageing water supply infrastructure, severe weather events 

These similarities shows an overlap in some of the risk factors (e.g., regulations in general might include environmental regulations). Additionally, it tells us how one risk may lead another to another, and with this information we might be able to predict which other risk to water supply may be also a problem if another exist already in a given state. 

In [16]:
#delete everything 
query = """
match(n) detach delete n"""

nodes= graph.run(query)

This concludes the first bit of the data exploration with Neo4j. In part 2.2, we will further explore the data by means of pie charts and maps. See you there!